<a href="https://colab.research.google.com/github/rshamsini/COVID-19detection/blob/main/label_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mounting the Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np #mathematical operations over arrays
import pandas as pd #a powerful data analysis and manipulation library for Python
import os
import glob #glob module is used to retrieve files/pathnames matching a specified pattern.
from pathlib import Path

In [ ]:
#read train_image_level.csv

data = pd.read_csv('/content/drive/MyDrive/Optum/train_image_level.csv')

In [ ]:
data.head()

,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [ ]:
#remove _image from id

for i in range(len(data)):
    t=data.loc[i,"id"]
    data.loc[i,"id"]=t[:-6]

In [ ]:
data.head()

,id,boxes,label,StudyInstanceUID
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [ ]:
#read train_study_level.csv

data2 = pd.read_csv('/content/drive/MyDrive/Optum/train_study_level.csv')

In [ ]:
data2.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1


In [ ]:
#remove _study from id

for i in range(len(data2)):
    t=data2.loc[i,"id"]
    data2.loc[i,"id"]=t[:-6]

In [ ]:
data2.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852,0,1,0,0
1,000c9c05fd14,0,0,0,1
2,00292f8c37bd,1,0,0,0
3,005057b3f880,1,0,0,0
4,0051d9b12e72,0,0,0,1


In [ ]:
#merge dataframes data,data2 based on StudyInstanceUID in data and id in data2

df = pd.merge(left=data, right=data2, left_on='StudyInstanceUID', right_on='id')
df.head()

,id_x,boxes,label,StudyInstanceUID,id_y,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,5776db0cec75,0,1,0,0
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,ff0879eb20ed,1,0,0,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,9d514ce429a7,0,1,0,0
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,28dddc8559b2,0,0,0,1
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,dfd9fdd85a3e,0,1,0,0


In [ ]:
#create a new column result and store the result values
#0-->Negative for pnumonia
#1-->Typical Appearance
#2-->Indeterminate Appearance
#3-->Atypical Appearance

res=[]

for i in range(len(df)) :
  if(df.loc[i,"Negative for Pneumonia"]):
    res.append(0)
  elif(df.loc[i,"Typical Appearance"]):
    res.append(1)
  elif(df.loc[i,"Indeterminate Appearance"]):
    res.append(2)
  elif(df.loc[i,"Atypical Appearance"]):
    res.append(3)

df['result']=res

In [ ]:
#drop unneccesary columns

df=df.drop(['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance'],axis=1)

In [ ]:
df.head()

,id_x,boxes,label,StudyInstanceUID,id_y,result
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,5776db0cec75,1
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,ff0879eb20ed,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,9d514ce429a7,1
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,28dddc8559b2,3
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,dfd9fdd85a3e,1


In [ ]:
#copied df to data (just to have variable name as data in future use)

data=df

# Label creation for train image dataset

In [ ]:
path = '/content/drive/MyDrive/Optum/images/train'
 
d=Path(path)
# This would print all the files and directories
images=[]
for files in d.glob('*.png'):
  images.append(files)

In [ ]:
len(images)

4334

In [ ]:
#take out only the image id from the image path

list_images=[]

for img in images:
    ss=str(img)
    ss=ss[-16:]
    ss=ss[:-4]
    list_images.append(ss)

In [ ]:
#drop rows that does not have corresponding images

for i in range(len(data)):
    if data.loc[i,"id_x"] not in list_images:
       data=data.drop(i)

In [ ]:
data.head(5)

,id_x,boxes,label,StudyInstanceUID,id_y,result
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,5776db0cec75,1
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,ff0879eb20ed,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,9d514ce429a7,1
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,28dddc8559b2,3
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,dfd9fdd85a3e,1


In [ ]:
#after dropping reset the indices

data = data.reset_index()

In [ ]:
data.head(2)

,index,id_x,boxes,label,StudyInstanceUID,id_y,result
0,0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,5776db0cec75,1
1,1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,ff0879eb20ed,0


In [ ]:
len(data)

4334

In [ ]:
#image dependencies

import PIL
from PIL import Image

In [ ]:
#create labels for train images and store in the path specified as text files


path="/content/drive/MyDrive/Optum/labels/train/"
img_path = "/content/drive/MyDrive/Optum/images/train/"

for i in range(len(data)):
    f_name=data.loc[i,"id_x"]
    sample=data.loc[i,"label"]
    image=PIL.Image.open(img_path+f_name+".png")
    width,height=image.size
    #print(width,height)
    scale_x=(640/width)
    scale_y=(640/height)
    
    if sample[0] == 'o':
        sample=sample.split("opacity 1 ")
        res='1'
    else:
        sample=sample.split("none 1 ")
        res='0'
    content=""
    for j in range(1,len(sample)):
        temp=sample[j]
        temp=temp.split(" ")
        value=""
        x=int(np.round(float(temp[0])*scale_x,4))
        y=int(np.round(float(temp[1])*scale_y,4))
        w=int(np.round(float(temp[2])*scale_x,4))
        h=int(np.round(float(temp[3])*scale_y,4))
        w = w - x # xmax - xmin
        h = h - y # ymax - ymin
        xc = x + int(np.round(w/2)) # xmin + width/2
        yc = y + int(np.round(h/2)) # ymin + height/2
        value+=str(round(xc/640,6))+" "+str(round(yc/640,6))+" "+str(round(w/640,6))+" "+str(round(h/640,6))           
        content+=res+" "+value+"\n"
    f = open(path +f_name + ".txt", "w")
    f.write(content)
    f.close()
        


# Label creation for validation image dataset

In [ ]:
path = "/content/drive/MyDrive/Optum/images/val"
 
d=Path(path)
# This would print all the files and directories
images=[]
for files in d.glob('*.png'):
  images.append(files)

In [ ]:
#take out only the image id from the image path

list_images=[]

for img in images:
    ss=str(img)
    ss=ss[-16:]
    ss=ss[:-4]
    list_images.append(ss)

In [ ]:
df.head(2)

,id_x,boxes,label,StudyInstanceUID,id_y,result
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,5776db0cec75,1
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,ff0879eb20ed,0


In [ ]:
#drop rows that does not have corresponding images

for i in range(len(df)):
    if df.loc[i,"id_x"] not in list_images:
       df=df.drop(i)

In [ ]:
len(df)

1367

In [ ]:
#after dropping reset the indices

df = df.reset_index()

In [ ]:
df.head()

,index,id_x,boxes,label,StudyInstanceUID,id_y,result
0,23,cada5310214b,"[{'x': 981.75034, 'y': 256.08181, 'width': 225...",opacity 1 981.75034 256.08181 1207.54099 765.2...,1a58b43cf286,1a58b43cf286,1
1,28,cdd9e3aaf45a,NaN,none 1 0 0 1 1,74ba8f2badcb,74ba8f2badcb,0
2,32,b3ffe59e37c7,NaN,none 1 0 0 1 1,81f6b4b343f7,81f6b4b343f7,1
3,33,b43de320e7d9,"[{'x': 1148.33401, 'y': 1097.72709, 'width': 4...",opacity 1 1148.33401 1097.72709 1621.3605 1667...,81f6b4b343f7,81f6b4b343f7,1
4,143,b121806162c3,NaN,none 1 0 0 1 1,0fd2db233deb,0fd2db233deb,2


In [ ]:
#create labels for validation images and store in the path specified as text files

path="/content/drive/MyDrive/Optum/labels/val/"
img_path = "/content/drive/MyDrive/Optum/images/val/"

for i in range(len(df)):
    f_name=df.loc[i,"id_x"]
    sample=df.loc[i,"label"]
    image=PIL.Image.open(img_path+f_name+".png")
    width,height=image.size
    #print(width,height)
    scale_x=(640/width)
    scale_y=(640/height)
    
    if sample[0] == 'o':
        sample=sample.split("opacity 1 ")
        res='1'
    else:
        sample=sample.split("none 1 ")
        res='0'
    content=""
    for j in range(1,len(sample)):
        temp=sample[j]
        temp=temp.split(" ")
        value=""
        x=int(np.round(float(temp[0])*scale_x,4))
        y=int(np.round(float(temp[1])*scale_y,4))
        w=int(np.round(float(temp[2])*scale_x,4))
        h=int(np.round(float(temp[3])*scale_y,4))
        w = w - x # xmax - xmin
        h = h - y # ymax - ymin
        xc = x + int(np.round(w/2)) # xmin + width/2
        yc = y + int(np.round(h/2)) # ymin + height/2
        value+=str(round(xc/640,6))+" "+str(round(yc/640,6))+" "+str(round(w/640,6))+" "+str(round(h/640,6))+" "
        content+=res+" "+value+"\n"
    f = open(path +f_name + ".txt", "w")
    f.write(content)
    f.close()
        


In [ ]:
path = "/content/drive/MyDrive/Optum/labels/val"
 
d=Path(path)
# This would print all the files and directories
images=[]
for files in d.glob('*.txt'):
  images.append(files)

In [ ]:
len(images)

1367

In [ ]:
path = "/content/drive/MyDrive/Optum/labels/train"
 
d=Path(path)
# This would print all the files and directories
images=[]
for files in d.glob('*.txt'):
  images.append(files)

In [ ]:
len(images)

4334